# Steam game review analysis

## Imports

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import json
from tqdm import tqdm

debug = False
url = 'https://store.steampowered.com/search/?category1=998&supportedlang=english'

# URL contains # which the scraping loop will replace with the start value in each iteration
url_raw = 'https://store.steampowered.com/search/results/?query&start=#&count=50&dynamic_data=&sort_by=_ASC&category1=998&supportedlang=english&snr=1_7_7_230_7&infinite=1'
# url_raw = 'https://store.steampowered.com/search/results/?query&start=#&count=50&dynamic_data=&force_infinite=1&category1=998&supportedlang=english&filter=topsellers&ndl=1&snr=1_7_7_7000_7&infinite=1'

## Get number of games on page

In [2]:
def scrape_num_games(url):
    page = requests.get(url) # Get game homepage
    
    soup = BeautifulSoup(page.content, "html.parser")
    div = soup.find("div", {"id": "search_results_filtered_warning"})
    
    children = [str(i) for i in div.children]
    result = [i for i in children if '<div>' in i][0] # First child div contains game count
    game_count = result[result.index(">")+1:result.index(" ")].replace(",", "") # Get number of games returned by query
    excluded_game_count = result[result.index(". ")+2:result.index(" titles")].replace(",", "") # Get number of games not shown due to search preferences
    total_count = int(game_count) - int(excluded_game_count) # Calculate total number of games returned and shown

    return total_count

game_count = scrape_num_games(url)
print(f'[INFO] {game_count} games found') 

[INFO] 78912 games found


## Scrape game information

In [4]:
def scrape_game_info(records, url):
    page = requests.get(url)
    
    result = str(page.content.decode())
    
    string = '<span class=\"title\">Steam Deck<\/span>'
    results = re.findall(r'<a href=\\"https:\\\/\\\/store\.steampowered\.com\\\/app.{1,2750}\\r\\n\\t\\t\\t\\t', result)
    for i in range(len(results)):
        current = results[i]

        # DEBUGGING - Print the info about the current game to see the request content structure
        # print(current)

        try:

            # Get 'title'
            title = re.findall(r'class=\\"title\\">.{1,300}<\\\/span', current)[0]
            title = title[title.index(">")+1:title.index("<\/span>")]
    
            # Get 'prices'
            if re.search(r'class=\\"discount_prices\\">.{1,300}<\\\/div', current) == None: # No price information given
                discount_price = None
                final_price = None
            else:
                price = re.findall(r'class=\\"discount_prices\\">.{1,300}<\\\/div', current)[0] # All price information
                if "discount_original_price" in price: # The price of the game before discount
                    discount_price = price[price.index("original_price")+17:price.index("<\/div>")]
                    price = price[price.index("<\/div>")+7:]
                else: # There is no discount
                    discount_price = None
                if 'free' in price:
                    final_price = 'Free'
                else:
                    final_price = price[price.index('final_price\\')+14:price.index("<\/div>")] # The current price of the game, either regular or after discount
                if discount_price == None: # If no discount, set pre-discount price to same as current (final) price
                    discount_price = final_price
    
            # Get 'reviews'
            if re.search(r'class=\\"search_review_summary positive\\".{1,300}\\">', current) == None:
                review_class = None # Overwhelmingly positive, positive, mixed etc.
                reviews = None
                review_percent = None # Percent of reviews that are positive
                review_absolute = None # Number of reviews
            else:
                review = re.findall(r'class=\\"search_review_summary positive\\".{1,300}\\">', current)[0]
                review_class = review[review.index('html=\\"')+7:review.index("&lt")]
                reviews = review[review.index('&gt;')+4:review.index('.')]
                review_percent = reviews[:reviews.index('%')]
                review_absolute = reviews[reviews.index('the ')+4:reviews.index(' user')].replace(",", "")
                
            # Get 'date'
            if re.search(r'class=\\"col search_released responsive_secondrow\\">.{1,200}<\\\/div>', current) == None:
                date = None
            else:
                date = re.findall(r'class=\\"col search_released responsive_secondrow\\">.{1,200}<\\\/div>', current)[0]
                date = date[date.index(r'\r\n')+4:date.index('<\/div>')].lstrip().rstrip()
                if date == "": # If no date, will match to whitespace so will be stripped to empty string
                    date = None
    
            records.append([title, discount_price, final_price, review_percent, review_absolute, review_class, date])
        except:
            print('[INFO] Cannot parse following info.')
            print(current)
            print('=====')
            
    return records

# Scrapes game info in batches of 50, so total number of games scraped = ~50*lim
if debug:
    lim = 1
else:
    lim = game_count // 50
records = []
print('[INFO] Scraping... ')
for i in tqdm(range(lim)):
    records = scrape_game_info(records, url_raw.replace('#', str(i*50)))

df = pd.DataFrame(records, columns=['title', 'pre-discount-price', 'sale-price', 'positive-review-percent', 'num-reviews', 'review-class', 'date-released'])
df

[INFO] Scraping... 


 52%|█████████████████████████████████████████                                      | 819/1578 [07:26<17:54,  1.42s/it]

[INFO] Cannot parse following info.
<a href=\"https:\/\/store.steampowered.com\/app\/2238430\/MonYu_Defeat_Monsters_And_Gain_Strong_Weapons_And_Armor_You_May_Be_Defeated_But_Dont_Give_Up_Become_Stronger_I_Believe_There_Will_Be_A_Day_When_The_Heroes_Defeat_The_Devil_King\/?snr=1_7_7_230_150_819\"\r\n\t\t\t data-ds-appid=\"2238430\" data-ds-itemkey=\"App_2238430\" data-ds-tagids=\"[122,4434,1720,3871,1684,4747,4182]\" data-ds-crtrids=\"[33021927]\" onmouseover=\"GameHover( this, event, 'global_hover', {&quot;type&quot;:&quot;app&quot;,&quot;id&quot;:2238430,&quot;public&quot;:1,&quot;v6&quot;:1} );\" onmouseout=\"HideGameHover( this, event, 'global_hover' )\" class=\"search_result_row ds_collapse_flag \"\r\n\t\t   data-search-page=\"819\" data-gpnav=\"item\"\r\n\t\t   data-ds-steam-deck-compat-handled=\"true\">\r\n            <div class=\"col search_capsule\"><img src=\"https:\/\/cdn.cloudflare.steamstatic.com\/steam\/apps\/2238430\/capsule_sm_120.jpg?t=1695279689\" srcset=\"https:\/\/cd

100%|██████████████████████████████████████████████████████████████████████████████| 1578/1578 [16:36<00:00,  1.58it/s]


,title,pre-discount-price,sale-price,positive-review-percent,num-reviews,review-class,date-released
0,Lethal Company,£8.50,£8.50,98,148630,Overwhelmingly Positive,"23 Oct, 2023"
1,Baldur's Gate 3,£49.99,£49.99,96,447993,Overwhelmingly Positive,"3 Aug, 2023"
2,Counter-Strike 2,Free,Free,87,7794022,Very Positive,"21 Aug, 2012"
3,Call of Duty®,None,None,None,None,None,"27 Oct, 2022"
4,House Flipper 2,£32.49,£29.24,83,1792,Very Positive,"14 Dec, 2023"
...,...,...,...,...,...,...,...
77846,Fatermyth,None,None,None,None,None,January 2024
77847,Hypnofantasis,None,None,None,None,None,To be announced
77848,Cerulean Singe,None,None,None,None,None,To be announced
77849,GARBAGE COUNTRY,None,None,None,None,None,To be announced


## Change date format

In [5]:
from sklearn.preprocessing import LabelEncoder

# Change month into number format
def switch(date):
    if date == 'Jan':
        return '01'
    elif date == 'Feb':
        return '02'
    elif date == 'Mar':
        return '03'
    elif date == 'Apr':
        return '04'
    elif date == 'May':
        return '05'
    elif date == 'Jun':
        return '06'
    elif date == 'Jul':
        return '07'
    elif date == 'Aug':
        return '08'
    elif date == 'Sep':
        return '09'
    elif date == 'Oct':
        return '10'
    elif date == 'Nov':
        return '11'
    elif date == 'Dec':
        return '12'
    return None

# Change date format to YYYY-MM-DD
print('[INFO] Changing date format...')
for i in tqdm(range(len(df))):
    date = str(df['date-released'].iloc[i])
    if not date == 'None' and not '-' in date:
        try:
            year = date[date.index(", ")+2:]
            month = date[date.index(" ")+1:date.index(",")]
            day = date[:date.index(" ")]
            month = switch(month) # Turn month into number format
            date = year + '-' + month + '-' + day.zfill(2)
        except:
            date = 'None'
        df['date-released'].iloc[i] = date

df_raw = df

[INFO] Changing date format...


100%|███████████████████████████████████████████████████████████████████████████| 77851/77851 [03:02<00:00, 427.31it/s]


## Clean up data - remove missing values and change formatting

In [6]:
# Remove rows where review-class is None
df = df.loc[df['review-class'].notna()]

df = df.loc[df['sale-price'].notna()]
df = df.loc[df['pre-discount-price'].notna()]


for i in tqdm(range(len(df))):
    discount_price = df['pre-discount-price'].iloc[i]
    sale_price = df['sale-price'].iloc[i]

    if sale_price == 'Free':
        sale_price = '0.00'
        discount_price = '0.00'

    discount_price = discount_price.replace("£", "")
    sale_price = sale_price.replace("£", "")

    df['pre-discount-price'].iloc[i] = discount_price
    df['sale-price'].iloc[i] = sale_price

df

100%|███████████████████████████████████████████████████████████████████████████| 29209/29209 [00:54<00:00, 535.92it/s]


,title,pre-discount-price,sale-price,positive-review-percent,num-reviews,review-class,date-released
0,Lethal Company,8.50,8.50,98,148630,Overwhelmingly Positive,2023-10-23
1,Baldur's Gate 3,49.99,49.99,96,447993,Overwhelmingly Positive,2023-08-03
2,Counter-Strike 2,0.00,0.00,87,7794022,Very Positive,2012-08-21
4,House Flipper 2,32.49,29.24,83,1792,Very Positive,2023-12-14
5,Rust,34.99,17.49,87,820392,Very Positive,2018-02-08
...,...,...,...,...,...,...,...
34806,Shell's Kitchen,0.00,0.00,70,20,Mostly Positive,2023-11-24
34807,Retro Combat,0.00,0.00,71,38,Mostly Positive,2023-10-28
34808,Touhou Meijinka ~ Song of Divine Tempest,0.00,0.00,75,12,Mostly Positive,2023-12-07
34809,DOMINO The Little One,0.00,0.00,79,54,Mostly Positive,2023-11-23


## Save DataFrame to CSV

In [7]:
def save_df(df):
    df.to_csv('data.csv', header=False, index=False)

save_df(df)